# ** Exclusions **
This is a short noteboook that will calculate the exclusions from the aggregated bi-section task data. 
To run the the analysis click on each block of code and then click the run icon on the toolbar at the top. Alternatively, click a code block and press shift-enter. [Click for here for help](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Running%20Code.html)


*Refer to the [Exclusion criterion notebook](Exclusion criterion .ipynb) for a detailed walkthrough.*


In [46]:
import numpy as np
import pandas as pd 


import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

import math 
import pylab 
from scipy import stats

from statsmodels import robust

import colorama
from colorama import Fore

from IPython.core.display import display, HTML

<a id='1'></a>
## *Getting the data*
This section will get the data from your disk. Ensure that you fill the file path prompt correctly, otherwise there will be nothing to work with.

__[How to get a file-path on a mac](https://apple.stackexchange.com/questions/252171/mac-finder-getting-the-path-of-a-directory-or-file-as-as-string)__
<br> __[How to get a file-path on windows](https://stackoverflow.com/questions/32573080/how-can-i-get-the-path-to-a-file-in-windows-10)__

In [47]:
print("What condition is this?")
condition_name = input()
print("What is the file path?")
file_path = input()

What condition is this?
Gamma 4
What is the file path?
/Users/Akshi/Desktop/Correlation/Correlation_Analysis/Gamma_4.xlsm


Make sure the above file path is correct before running the next block of code

In [49]:
#Read the data *** Make sure path is set to the correct file-path *** 
path = file_path
data_sheet = pd.ExcelFile(path)
#Create a pdf
pdf = PdfPages("Analysis of "+ condition_name)

#Parse the Exclusions sheet to create a Pandas DataFrame
exlcusions = data_sheet.parse('1. Exclusions')
#Select the columns that are needed and create a new DataFrame with them
DF = exlcusions[["ID","subCondition","highRef","estimatedMid","lowRef","roundType", "AnchorValues"]]
#Drop NaN values
DF = DF.dropna(subset=["estimatedMid"])

#Group the DataFrame by subcondition
sub_cond_df = DF.groupby("subCondition")

<a id='2'></a>
## *Functions*
This code block contains all the functions that will be used to conduct the analysis. 


In [62]:
#Functions to transform the data. Either box-cox or cbrt transforms are applied after acconting for anchoring

def transform(sub_cond):
    '''A function that combines attempts for a subcondition, in order to account for anchoring.
        Returns a numpy array with transformed data. Resulting distribution should be Gaussian.
        
        @param sub_cond: subconditon that will be transformed
        @return uni_modal: np.array with transformed data'''
    
    #0 corresponds to first attempt 
    first_idx = 0 
    second_idx = 1
    third_idx = 2
    fourth_idx = 3 
    
    #Get estimatedMid column from DataFrame
    estimates = sub_cond['estimatedMid']
    
    #Create a np-array for transformed data
    uni_modal= np.empty(int(len(sub_cond)/4))
    
    for i in range(int(len(sub_cond)/4)):
        #Get attempts for a participant
        first_atmpt = estimates.iloc[first_idx]
        second_atmpt = estimates.iloc[second_idx]
        third_atmpt = estimates.iloc[third_idx]
        fourth_atmpt = estimates.iloc[fourth_idx]
        #Calculate new estimate via Spencers suggested formula
        estimate = abs(((first_atmpt+third_atmpt) - (second_atmpt+fourth_atmpt)))
        #Add to np-array
        uni_modal[i] = estimate
        #Increase index to next participant
        first_idx+=4
        second_idx+=4
        third_idx+=4
        fourth_idx+=4
    
        
    #return transformed data 
    return uni_modal



def cbrt(data):
    '''function that applys a cubroot transform and returns the array
        @param data : array of estimates that are going to be transformed
        @return measurements: array with transforemd data '''
    
    #Apply cubroot transform 
    measurements = (measurements**(1/3))
     
    #return transformed data 
    return measurements


def box_cox(sub_cond):
    '''function that preforms box-cox transform and returns the array
        @param data : array of estimates that are going to be transformed
        @return measurements: array with transforemd data '''
    
    #Apply cubroot transform 
    measurements = stats.boxcox(measurements, 0)

    #return transformed data 
    return measurements


#End of transformation functions
#----------------------------------------------------------------------------------------------------------------
#A function that tests for normality of data

def norm_test(data, alpha=0.05):
    '''function that determines if given data is normal or not
    @param data: array containg the data that will have K^2 test applied to it
    @param alpha: significane level (default is 0.05)
    @return normal: boolean signifying if data is normally distributed or not'''
    
    normal = False 
    #K^2 test
    stat, p = stats.normaltest(measurements)
    
    #Print results
    print(Fore.BLACK+'Statistics=%.3f, p=%.3f' % (stat, p))
    
    # interpret p value
    alpha_val = 0.05
    if p > alpha_val:
        print(Fore.BLACK+ 'Sample looks Gaussian (fail to reject H0)')
        normal = True
    else:
        print(Fore.RED + 'SAMPLE NOT GAUSSIAN!!!!'  + '(reject H0)')
    print(Fore.BLACK + "---------------------------------")
    
    return normal

#----------------------------------------------------------------------------------------------------------------
#Functions to get Robust score and exclusions

def get_score(data):
    '''Function to calculate RobustScore as defined as: RS = (x - median)/MAD, where MAD is Medium Absolutle Deviation
        @param data: array for which score will be calculated '''
    
    score_list = []
    
    #calculate MAD and median
    mad = robust.scale.mad(data)
    median = np.median(data)
    
    for i in range(len(data)):
        #Calculate score for each data point
        num = (data[i]-median)
        denom = mad
        score = num/denom
        #add to list
        score_list.append(score)
        
    return score_list

def exclusion(data):
    '''Function that calculate the exclusions for an array and returns 
        @param data :data for which exclusions will get calculated
        @return exclusions_idx: array containg IDs of participants that should be excluded'''
    
    #exclusions_idx contains the indicies of any participants for the given subconditon that should be excluded 
    exclusions_idx = []

    #get robust score
    data_score = np.abs(np.array(get_score(data)))
    
    #get indicies of exclusions
    exclusions_idx = np.where(data_score > 2.5)[0].tolist()
    
    #increment index to match participant IDs
    map(lambda x:x+1, exclusions_idx)

    return exclusions_idx 

        
#----------------------------------------------------------------------------------------------------------------

def plotter(measurements):
    '''Function to plot distribution and Normal QQ to the pdf
        @param measurements: np array of vlaues that will be plotted'''
    
    #plot histogram
    plt.subplot(1,2,1)
    sns.distplot(measurements, kde=False)
    plt.title("Subcondition " + str(i) + " transformed distribution")
    #Normal QQ plot
    plt.subplot(1,2,2)
    stats.probplot(measurements, dist="norm", plot=plt)
    plt.title("Subcondition " + str(i) + " QQ plot")
    plt.show()



# IGNORE STUFF BELOW THIS
# THIS IS STILL INCOMPLETE

## *Transforming the data*
This section will transform the data amd determine if the sample is normally distributed. The most important sub-condition to look at is the first one. If it does not look Gaussian a different transformation may need to be applied. Consult Ron or a project leader before continuing. 

In [51]:

def norm_test(data, alpha=0.05):
    '''function that determines if given data is normal or not
    @param data: array containg the data that will have K^2 test applied to it
    @param alpha: significane level (default is 0.05)
    @return normal: boolean signifying if data is normally distributed or not'''
    
    normal = False 
    #K^2 test
    stat, p = stats.normaltest(measurements)
    
    #Print results
    print(Fore.BLACK+'Statistics=%.3f, p=%.3f' % (stat, p))
    
    # interpret p value
    alpha_val = 0.05
    if p > alpha_val:
        print(Fore.BLACK+ 'Sample looks Gaussian (fail to reject H0)')
        normal = True
    else:
        print(Fore.RED + 'SAMPLE NOT GAUSSIAN!!!!'  + '(reject H0)')
    print(Fore.BLACK + "---------------------------------")
    
    return normal

Subcondition 1 normaility test: 
Statistics=5.623, p=0.060
Sample looks Gaussian (fail to reject H0)
---------------------------------
Subcondition 2 normaility test: 
Statistics=2.145, p=0.342
Sample looks Gaussian (fail to reject H0)
---------------------------------
Subcondition 3 normaility test: 
Statistics=12.695, p=0.002
SAMPLE NOT GAUSSIAN!!!!(reject H0)
---------------------------------
Subcondition 4 normaility test: 
Statistics=2.487, p=0.288
Sample looks Gaussian (fail to reject H0)
---------------------------------
Subcondition 5 normaility test: 
Statistics=0.467, p=0.792
Sample looks Gaussian (fail to reject H0)
---------------------------------
Subcondition 6 normaility test: 
Statistics=1.396, p=0.498
Sample looks Gaussian (fail to reject H0)
---------------------------------
Subcondition 7 normaility test: 
Statistics=2.454, p=0.293
Sample looks Gaussian (fail to reject H0)
---------------------------------
Subcondition 8 normaility test: 
Statistics=1.472, p=0.479
Sa